In [14]:
!pip install matplotlib

  Using cached matplotlib-3.10.6-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.6-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl (226 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 16.0 MB/s eta 0:00:00
Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl (73 kB)
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)


In [3]:
import tensorflow as tf
import glob
import os
import numpy as np

In [10]:
images = r"D:\WORK\Python\Project\CV\flood_map_segmentation\floodmap_segmentation\data\Image\*.jpg"
masks = r"D:\WORK\Python\Project\CV\flood_map_segmentation\floodmap_segmentation\data\Mask\*.png"

len(images), len(masks)

(87, 86)

In [15]:
import cv2
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
class CustomDataGenerator(keras.utils.Sequence):
    def __init__(self, images, masks, batch_size=8, img_size=(512, 512), shuffle=True):
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.images = images   # list of image file paths
        self.masks = masks     # list of mask file paths

        self.on_epoch_end()
        
        self.datagen = ImageDataGenerator(
            rotation_range=5,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='constant',
            cval=0.0
        )



    def random_crop(self, image, mask, crop_size=512):
        img_height, img_width = image.shape[0], image.shape[1]
        mask_height, mask_width = mask.shape[0], mask.shape[1]
        
        
        # Random x and y for cropping the image
        x = np.random.randint(0, img_width - crop_size)
        y = np.random.randint(0, img_height - crop_size)
        
        image_corpped = image[y:y+crop_size, x:x+crop_size]
        mask_crop = mask[y:y+crop_size, x:x+crop_size]
        
        return image_corpped, mask_crop
        
        
    def data_augmentation(self, image, mask):
       

    def __len__(self):
        """
        Number of batches per epoch.
        """
        return int(np.ceil(len(self.images) / float(self.batch_size)))

    def data_normalization(self, image, mask):
        """
        Normalize image and mask.
        """
        pass
    
    def data_preprocessing(self, image, mask):
        pass

    def on_epoch_end(self):
        pass
    

    def __getitem__(self, index):
        """
        Generate one batch of data.
        """
        start_index = index * self.batch_size
        end_index = min((index + 1) * self.batch_size, len(self.images))
        indexes = self.indexes[start_index:end_index]
        
        batch_images = []
        batch_masks = []